# Classification Project - Fashion MNIST (CNN)

## Description:

The datasets consists of 60,000 images and each image has 784 features. An image consists of 28x28 pixels, and each pixel is a value from 0 to 255 describing the pixel intensity. 0 for white and 255 for black.

The class labels for Fashion MNIST are:

    Label     Description
    ---------------------
    0         T-shirt/top
    1         Trouser 
    2         Pullover 
    3         Dress
    4         Coat
    5         Sandal 
    6         Shirt
    7         Sneaker
    8         Bag
    9         Ankle boot

In [1]:
# Importing the dependencies

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import gzip

%matplotlib inline

In [2]:
# Function to reset graph and give default seed

def reset_graph(seed=20):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
# Function to display image in the dataset

def showImage(data, color=True):
    some_article_image = data.reshape(28,28)
    cmap = plt.cm.binary if color==False else None
    plt.imshow(some_article_image, cmap=cmap, interpolation='nearest')
    plt.axis('off')
    plt.show()

# Importing dataset

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
fashion_mnist = input_data.read_data_sets("/cxldata/datasets/project/fashion-mnist")

W0826 10:30:56.150416 140680462427968 deprecation.py:323] From <ipython-input-4-975a031a623b>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0826 10:30:56.151747 140680462427968 deprecation.py:323] From /usr/local/anaconda/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0826 10:30:56.154238 140680462427968 deprecation.py:323] From /usr/local/anaconda/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future ve

Extracting /cxldata/datasets/project/fashion-mnist/train-images-idx3-ubyte.gz


W0826 10:30:56.507643 140680462427968 deprecation.py:323] From /usr/local/anaconda/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0826 10:30:56.574107 140680462427968 deprecation.py:323] From /usr/local/anaconda/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting /cxldata/datasets/project/fashion-mnist/train-labels-idx1-ubyte.gz
Extracting /cxldata/datasets/project/fashion-mnist/t10k-images-idx3-ubyte.gz
Extracting /cxldata/datasets/project/fashion-mnist/t10k-labels-idx1-ubyte.gz


# Defining Deep Convolutional Neural Network

In [ ]:
# Defining parameters for training

height = 28
width = 28
channels = 1
n_inputs = height * width

conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 64
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"

pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = 10

Input --> CN1 --> CN2 --> Max Pool --> FC1 --> Output

In [5]:
# Defining CNN

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X") # None, 784
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels]) # None, 28, 28, 1
    y = tf.placeholder(tf.int32, shape=[None], name="y") # [None]

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")

# conv1 -> None, 28, 28, 32
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")


# conv2 -> None, 14, 14, 64
with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    # pool3 -> None, 7, 7, 64
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 7 * 7])
    #pool3_flat -> None, 64*7*7

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init"):
    init = tf.global_variables_initializer()

W0826 10:31:04.659290 140680462427968 deprecation.py:323] From <ipython-input-5-2017e4f1d53c>:30: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0826 10:31:04.664551 140680462427968 deprecation.py:506] From /usr/local/anaconda/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0826 10:31:04.896965 140680462427968 deprecation.py:323] From <ipython-input-5-2017e4f1d53c>:48: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


# Training and evaluating CNN

In [6]:
n_epochs = 10
batch_size = 100

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(fashion_mnist.train.num_examples // batch_size):
            X_batch, y_batch = fashion_mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: fashion_mnist.test.images, y: fashion_mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

0 Train accuracy: 0.9 Test accuracy: 0.8695
1 Train accuracy: 0.92 Test accuracy: 0.8909
2 Train accuracy: 0.89 Test accuracy: 0.8986
3 Train accuracy: 0.89 Test accuracy: 0.9065
4 Train accuracy: 0.92 Test accuracy: 0.9071
5 Train accuracy: 0.94 Test accuracy: 0.9118
6 Train accuracy: 0.96 Test accuracy: 0.9107
7 Train accuracy: 0.96 Test accuracy: 0.9086
8 Train accuracy: 0.97 Test accuracy: 0.9134
9 Train accuracy: 0.96 Test accuracy: 0.9106


<div class="alert alert-block alert-info" style="margin-top: 0px">

Project prepared by:

**Dhyey Kotecha**

Email: dhyeygkotecha@gmail.com

(Aug 26, 2020)